(ch:working_with_keras)=
# 케라스 신경망 모델 활용법

**감사의 글**

아래 내용은 프랑소와 숄레의 
[Deep Learning with Python(2판)](https://github.com/fchollet/deep-learning-with-python-notebooks)의 
소스코드 내용을 참고해서 작성되었습니다.
자료를 공개한 저자에게 진심어린 감사를 전합니다.

**소스코드**

여기서 언급되는 코드를
[(구글 코랩) 신경망 모델 활용법](https://colab.research.google.com/github/codingalzi/dlp2/blob/master/notebooks/NB-working_with_keras.ipynb)에서 
직접 실행할 수 있다.

**슬라이드**

본문 내용을 요약한 [슬라이드](https://github.com/codingalzi/dlp2/raw/master/slides/slides-working_with_keras.pdf)를 다운로드할 수 있다.

**주요 내용**

- 다양한 신경망 모델 구성법
- 신경망 모델과 층의 재활용
- 신경망 모델 훈련 옵션: 콜백과 텍서보드

## 신경망 모델 구성법 1: `Sequential` 모델 활용

케라스를 이용하여 매우 간단한 방식부터 매우 복잡한 방식까지 다양한 방식으로 
필요한 수준의 신경망 모델을 구성할 수 있다.
케라스를 이용하여 세 가지 방식으로 딥러닝 신경망 모델을 구성할 수 있다.

- `Sequential` 모델 활용: 층으로 스택을 쌓아 만든 모델
- 함수형 API 활용: 가장 많이 사용됨.
- 모델 서브클래싱: 모든 것을 사용자가 지정.

먼저 `Sequential` 모델은 층으로 스택을 쌓아 만든 모델이며 가장 단순하다.

**`Sequential` 클래스**

```python
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(64, activation="relu"),
    layers.Dense(10, activation="softmax")
])
```

`Sequential` 모델은 한 종류의 입력값과 한 종류의 출력값만 사용 가능하며, 데이터 변환은 지정된 층의 순서대로 적용된다.

**`summary()` 메서드**

완성된 신경망 모델의 층의 구조와 각 층의 출력값의 모양 정보를 확인할 수 있다.

- 모델과 층의 이름: 특별히 지정하지 않으면 기본 이름 자동으로 할당됨.
- 층별 출력값의 모양(`Output Shape`): 유닛수 확인. `None` 은 임의의 배치크기를 가리킴.
- 층별 파라미터 수(`Param #`): 훈련을 통해 학습되어야 하는 파라미터들의 개수
    - 가중치 행렬과 편향 벡터에 포함된 항목들의 수를 가리킴.
- 파라미터 수
    - `Total params`: 파라미터의 총 개수
    - `Trainable params`: 가중치 행렬과 편향 벡터에 포함된 파라리터처럼 훈련을 통해 학습(업데이트)되는 파라미터 총 개수
    - `Non-trainable params`: 훈련에는 관여하지만 훈련을 통해 학습되지는 않는 파라미터. 
        지금까지 다룬 모델에서는 비학습 파라미터 사용되지 않았음.
        나중에 비학습 파라미터 예제를 다룰 것임 ({numref}`%s장 <ch:computer-vision-advanced>` 참고).

그런데 위 모델의 요약이 정보를 제대로 전달하지 못한다.

```python
>>> model.summary()
Model: "sequential"
┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                   ┃ Output Shape           ┃      Param # ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ dense (Dense)                  │ ?                      │  0 (unbuilt) │
├────────────────┼────────────┼───────┤
│ dense_1 (Dense)                │ ?                      │  0 (unbuilt) │
└────────────────┴────────────┴───────┘
 Total params: 0 (0.00 B)
 Trainable params: 0 (0.00 B)
 Non-trainable params: 0 (0.00 B)
```

이유는 모델의 입력값의 모양을 모르기 때문이다.
사실 위 모델은 임의의 모양의 2차원 텐서를 입력값으로 사용할 수 있다.

**`Input()` 함수**

`Input()`함수를 이용하여 모델 훈련에 사용될 입력값에 대한 정보를 모델에 미리 전달할 수 있다. 
예를 들어 아래 코드는 훈련셋의 샘플이 784 개의 특성을 갖는 1차원 어레이로 지정한다.
즉, 이렇게 입력값의 정보를 미리 지정하면 지정된 모양의 훈련셋만 사용할 수 있다.

```python
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    keras.Input(shape=(784,)),
    layers.Dense(64, activation="relu"),
    layers.Dense(10, activation="softmax")
])
```

그러면 모델 정보가 보다 구체적으로 요약된다.

```python
>>> model.summary()
Model: "sequential"
┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                   ┃ Output Shape           ┃      Param # ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ dense (Dense)                  │ (None, 64)             │       50,240 │
├────────────────┼────────────┼───────┤
│ dense_1 (Dense)                │ (None, 10)             │          650 │
└────────────────┴────────────┴───────┘
 Total params: 50,890 (198.79 KB)
 Trainable params: 50,890 (198.79 KB)
 Non-trainable params: 0 (0.00 B)
```

**층별 가중치 텐서**

위 모델에 대해서 층별로 가중치 행렬과 편향 벡터 하나씩 총 4 개의 텐서가 생성된다.
생성된 모든 텐서를 항목으로 갖는 리스트를 모델의 `weights` 속성이 가리킨다.

```python
>>> model.weights
[<KerasVariable shape=(784, 64), dtype=float32, path=sequential_2/dense_4/kernel>,
 <KerasVariable shape=(64,), dtype=float32, path=sequential_2/dense_4/bias>,
 <KerasVariable shape=(64, 10), dtype=float32, path=sequential_2/dense_5/kernel>,
 <KerasVariable shape=(10,), dtype=float32, path=sequential_2/dense_5/bias>]
```

- 1층의 가중치 행렬(2차원 텐서) 모양: (784, 64)
    - 입력값 특성: 784개
    - 출력값 특성: 64개

```python
>>> model.weights[0].shape # 가중치 행렬
TensorShape([784, 64])
```

- 1층의 편향 벡터(1차원 텐서) 모양: (64,)
    - 출력값 특성: 64개    

```python
>>> model.weights[1].shape # 편향 벡터
TensorShape([64])
```

- 2층의 가중치 행렬(2차원 텐서): (64, 10)
    - 입력값 특성: 64개
    - 출력값 특성: 10개

```python
>>> model.weights[2].shape # 가중치 행렬
TensorShape([64, 10])
```

- 2층의 편향 벡터(1차원 텐서) 모양: (10,)
    - 출력값 특성: 10개    

```python
>>> model.weights[3].shape # 편향 벡터
TensorShape([10])
```

## 신경망 모델 구성법 2: 함수형 API

`Sequential` 클래스를 사용하면 한 종류의 입력값과 한 종류의 출력값만을 사용할 수 있다.
반면에 함수형 API를 이용하면 다중 입력과 다중 출력을 지원하는
모델을 구성할 수 있다.
즉, 모델이 서로 다른 종류의 입력값과 서로 다른 종류의 출력값을 지원한다.

**기본 활용법**

앞서 살펴 본 `Sequential` 모델을 함수형 API를 이용하여 구성하면 다음과 같다.

```python
inputs = keras.Input(shape=(3,), name="my_input")          # 입력층
features = layers.Dense(64, activation="relu")(inputs)     # 은닉층
outputs = layers.Dense(10, activation="softmax")(features) # 출력층
model = keras.Model(inputs=inputs, outputs=outputs)        # 모델 지정
```

사용된 단계들을 하나씩 살펴보자.

- 입력층: 여기서는 한 개만 사용

    ```python
    inputs = keras.Input(shape=(3,), name="my_input")
    ```

- 은닉층: 입력층의 값을 이어 받음. 여러 개의 은닉층을 자유 자재로 구성 가능.

    ```python
    features = layers.Dense(64, activation="relu")(inputs)
    ```

- 출력층: 최종 은닉층의 결과를 이어 받아 출력값 계산. 여기서는 한 개만 사용.

    ```python
    outputs = layers.Dense(10, activation="softmax")(features)
    ```

- 모델 빌드: 입력층과 출력층을 이용하여 모델 지정

    ```python
    model = keras.Model(inputs=inputs, outputs=outputs)
    ```

`summary()` 의 실행결과는 이전과 동일하다.

```python
>>> model.summary()
Model: "functional_1" 
_________________________________________________________________
Layer (type)                 Output Shape              Param # 
=================================================================
my_input (InputLayer)        [(None, 3)]               0 
_________________________________________________________________
dense_6 (Dense)              (None, 64)                256 
_________________________________________________________________
dense_7 (Dense)              (None, 10)                650 
=================================================================
Total params: 906 
Trainable params: 906 
Non-trainable params: 0 
_________________________________________________________________
```

### 다중 입력, 다중 출력 모델

다중 입력과 다중 출력을 지원하는 모델을 구성하는 방법을 예제를 이용하여 설명한다.

:::{prf:example} 고객 요구사항 접수 모델
:label: exp-customer

고객의 요구사항이 입력되었을 때 처리 우선순위와 담당부서를 지정하는 시스템을 구현하려 한다.
고객의 요구사항은 세 종류의 입력값으로 구성된다.
따라서 시스템에 사용될 딥러닝 신경망 모델은 세 개의 입력값과 두 개의 출력값을 사용한다. 

입력 사항 세 종류는 다음과 같다.

- `title`: 요구사항 제목
    - 제목을  멀티-핫 인코딩한 0과 1로 구성된 길이가 1만인 벡터. 
    - 1만등 이내의 사용빈도를 갖는 단어만 사용됨.
    - {numref}`%s장 <ch:classification_regression>`의IMDB 영화 후기 멀티-핫 인코딩 참고
- `text_body`: 요구사항 내용
    - 요구사항 내용을  멀티-핫 인코딩한 0과 1로 구성된 길이가 1만인 벡터. 
    - 1만등 이내의 사용빈도를 갖는 단어만 사용됨.
    - {numref}`%s장 <ch:classification_regression>`의IMDB 영화 후기 멀티-핫 인코딩 참고
- `tags`: 사용자에 의한 추가 선택 사항
    - 100개 중 여러 개 선택. 
    - 멀티-핫 인코딩된 길이가 1만인 벡터.

출력 사항 두 종류는 다음과 같다.

- `priority`: 요구사항 처리 우선순위. 
    - 0에서 1사이의 값. 
    - `sigmoid` 활성화 함수 활용.
- `department`: 네 개의 요구사항 처리 담당 부서 중 하나 선택. 
    - `softmax` 활성화 함수 활용.
:::

고객 요구사항 접수 모델을 함수형 API를 이용하여 다음과 같이 
구현할 수 있다.

- 입력층: 세 개
    - `title`: 제목 입력 처리
    - `text_body`:  요구사항 내용 입력 처리
    - `tags`: 추가 선택사항 처리

- 은닉층: 두 개
    - `Concatenate` 층: 여러 개의 텐서를 좌우로 이어붙임. 
        여기서는 고객의 요구사항 제목, 내용, 추가 선택사항을 담고 있는
        길이가 각각 1만, 1만, 100인 세 개의 벡터를 이어붙여서 
        길이가 총 2만1백인 하나의 벡터로 변환.
    - `Dense` 층: 64 개의 유닛과 함께 `relu()` 함수 사용
    
- 출력층: 두 개
    - `priority`: 요구사항 처리 우선순위 판단. 0과 1사의 값을 계산하는 `Dense` 층. 
        1개의 유닛과 `sigmoid()` 활성화함수 지정.
    - `department`: 요구사항 처리 담당 부서 선택. 4 개의 부서 각각에 대한 처리 활률값을 계산하는 `Dense` 층. 
        4개의 유닛과 `softmax()` 활성화함수 지정.

```python
vocabulary_size = 10000    # 사용빈도 1만등 인내 단어 사용
num_tags = 100             # 태그 수
num_departments = 4        # 부서 수

# 입력층: 세 개
title = keras.Input(shape=(vocabulary_size,), name="title")
text_body = keras.Input(shape=(vocabulary_size,), name="text_body")
tags = keras.Input(shape=(num_tags,), name="tags")

# 은닉층
features = layers.Concatenate()([title, text_body, tags]) # shape=(None, 10000+10000+100)
features = layers.Dense(64, activation="relu")(features)

# 출력층: 두 개
priority = layers.Dense(1, activation="sigmoid", name="priority")(features)
department = layers.Dense(
    num_departments, activation="softmax", name="department")(features)

# 모델 빌드: 입력값으로 구성된 입력값 리스트와 출력값으로 구성된 출력값 리스트 사용
model = keras.Model(inputs=[title, text_body, tags], outputs=[priority, department])
```

**모델 컴파일**

손실함수와 평가지표 모두 출력층의 개수만큼 지정한다.

- 손실함수(loss)
    - `priority` 대상: `mean_squared_error`
    - `department` 대상: `categorical_crossentropy`

- 평가지표(metrics): 평가지표는 대상 별로 여러 개를 사용할 수 있기에 각각의 대상에 대해 평가지표 리스트를 작성함.
    - `priority` 대상: `["mean_absolute_error", "mean_squared_error"]`
    - `department` 대상: `["accuracy", "AUC", "Precision"]`

```python
model.compile(optimizer="adam",
              loss=["mean_squared_error", "categorical_crossentropy"],
              metrics=[["mean_absolute_error", "mean_squared_error"], ["accuracy", "AUC", "Precision"]])
```

**모델 훈련**

모델 훈련은 `fit()` 함수에 세 개의 훈련 텐서로 이루어진 리스트와 
두 개의 타깃 텐서로 이루어진 리스트를 지정한 후에 실행한다. 

```python
model.fit([title_data, text_body_data, tags_data],
          [priority_data, department_data]
          epochs=10)
```

**모델 평가**

모델 평가도 훈련과 동일한 방식으로 구성된 인자가 사용된다.

```python
model.evaluate([title_data, text_body_data, tags_data],
               [priority_data, department_data])
```

**모델 활용**

예측값은 두 개의 출력층 각각에서 생성된 두 개의 어레이로 구성된 리스트이다.

```python
priority_preds, department_preds = model.predict([title_data, text_body_data, tags_data])
```

- 우선 순위 예측값: 0과 1사이의 확률값

    ```python
    >>> priority_preds
    array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)
    ```

- 처리 부서 예측값: 각 부서별 적정도를 가리키는 확률값

    ```python
    >>> department_preds
    array([[1.26754885e-05, 3.67883259e-11, 2.38737906e-03, 9.97599900e-01],
           [5.86307794e-03, 4.93278662e-09, 6.00390911e-01, 3.93745989e-01],
           [1.56256149e-03, 3.38436678e-07, 2.20820252e-02, 9.76355135e-01],
           ...,
           [2.97836447e-03, 6.37571304e-07, 4.77804057e-03, 9.92242992e-01],
           [2.41168109e-05, 3.63892028e-10, 3.09850991e-01, 6.90124929e-01],
           [9.11577154e-05, 7.13576198e-10, 7.34233633e-02, 9.26485479e-01]],
           dtype=float32)
    ```

    각각의 요구사항을 처리해야 하는 부서는 `argmax()` 메서드로 확인된다.

    ```python
    >>> department_preds.argmax()
    array([3, 2, 3, ..., 3, 3, 3])
    ```

### 신경망 모델 구조 그래프

`plot_model()`을 이용하여 층 연결 구조를 그래프로 나타낼 수 있다.

```python
>>> keras.utils.plot_model(model, "ticket_classifier.png")
```

<div align="center"><img src="https://github.com/codingalzi/dlp2/blob/master/jupyter-book/imgs/ch07-ticket_classifier.png?raw=true" style="width:400px;"></div>

:::{admonition} 층 연결 그래프 그리기
:class: info

`keras.utils.plot_model()` 함수가 제대로 작동하려면 
`pydot` 파이썬 모듈과 graphviz 라는 프로그램이 컴퓨터에 설치되어 있어야 한다.

- `pydot` 모듈 설치: `pip install pydot`
- graphviz 프로그램 설치: [https://graphviz.gitlab.io/download/](https://graphviz.gitlab.io/download/)
- 구글 코랩에서는 기본으로 지원됨.
:::

입력 텐서와 출력 텐서의 모양을 함께 표기할 수도 있다.

```python
>>> keras.utils.plot_model(model, "ticket_classifier_with_shape_info.png", show_shapes=True)
```

<div align="center"><img src="https://github.com/codingalzi/dlp2/blob/master/jupyter-book/imgs/ch07-ticket_classifier_with_shapes.png?raw=true" style="width:900px;"></div>

### 신경망 모델 재활용

훈련된 모델을 새로운 모델을 구성하는 데에 활용할 수 있다.
먼저 모델의 `layers` 속성을 이용하여 사용된 층에 대한 정보를 확인한다. 
`layers` 속성은 사용된 층들로 이루어진 리스트를 가리킨다.

```python
>>> model.layers
[<keras.src.engine.input_layer.InputLayer at 0x7fc3a1313fd0>,
 <keras.src.engine.input_layer.InputLayer at 0x7fc3a13ce450>,
 <keras.src.engine.input_layer.InputLayer at 0x7fc3a13c5990>,
 <keras.src.layers.merging.concatenate.Concatenate at 0x7fc3a13e0d50>,
 <keras.src.layers.core.dense.Dense at 0x7fc3a13a6310>,
 <keras.src.layers.core.dense.Dense at 0x7fc3a12f6850>,
 <keras.src.layers.core.dense.Dense at 0x7fc3a13e2f90>]
```

예를 들어, 3번 인덱스에 해당하는 층의 입력값과 출력값에 대한 정보는 아래처럼 확인할 수 있다.

```python
>>> model.layers[3].input
[<KerasTensor: shape=(None, 10000) dtype=float32 (created by layer 'title')>,
 <KerasTensor: shape=(None, 10000) dtype=float32 (created by layer 'text_body')>,
 <KerasTensor: shape=(None, 100) dtype=float32 (created by layer 'tags')>]

>>> model.layers[3].output
<KerasTensor: shape=(None, 20100) dtype=float32 (created by layer 'concatenate')>
```

여기서는 문제해결의 어려움 정도를 "quick", "medium", "difficult"로 구분하는 어려움 정도를 판별하는
출력층을 추가하여 총 세 종류의 출력값을 생성하는 모델을 얻고자 한다.
원래는 모델을 새로 구성하고 훈련을 처음부터 다시 시작해야 한다.
하지만 어차피 훈련과정이 이전과 특별히 다를 이유가 없다.
이런 경우 이전에 훈련된 모델의 일부를 재활용할 수 있다.
기존에 훈련된 모델을 재활용하는 경우 
일반적으로 출력층을 제외한 나머지 층을 재활용 대상으로 삼는다.

기존 모델에서 출력층은 5번과 6번 인덱스에 위치하기에 
4번 인덱스가 가리키는 (은닉)층의 출력 정보를 따로 떼어낸다.

```python
>>> features = model.layers[4].output
```

그런 다음 이제 출력층에 문제해결의 어려움 정도를 판별하는 층을 아래와 같이 지정한다.

```python
>>> difficulty = layers.Dense(3, activation="softmax", name="difficulty")(features)
```

새로이 준비된 `'difficulty'` 층을 기존에 사용된 두 개의 출력층과 함께
새로운 모델의 출력층으로 지정한다.

```python
new_model = keras.Model(
    inputs=[title, text_body, tags],
    outputs=[priority, department, difficulty])
```

모델 구성 그래프는 다음과 같다.

```python
>>> keras.utils.plot_model(new_model, "updated_ticket_classifier.png", show_shapes=True)
```

<div align="center"><img src="https://github.com/codingalzi/dlp2/blob/master/jupyter-book/imgs/ch07-ticket_classifier_with_shapes-02.png?raw=true" style="width:900px;"></div>

새로 생성된 모델은 출력층을 제외한 나머지 층들이 기존 모델의 훈련 과정에서 학습한
가중치 행렬과 편향 벡터를 그대로 유지한다.

```python
>>> new_model.summary()
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
==================================================================================================
title (InputLayer)              [(None, 10000)]      0                                            
__________________________________________________________________________________________________
text_body (InputLayer)          [(None, 10000)]      0                                            
__________________________________________________________________________________________________
tags (InputLayer)               [(None, 100)]        0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 20100)        0           title[0][0]                      
                                                                 text_body[0][0]                  
                                                                 tags[0][0]                       
__________________________________________________________________________________________________
dense_8 (Dense)                 (None, 64)           1286464     concatenate[0][0]                
__________________________________________________________________________________________________
priority (Dense)                (None, 1)            65          dense_8[0][0]                    
__________________________________________________________________________________________________
department (Dense)              (None, 4)            260         dense_8[0][0]                    
__________________________________________________________________________________________________
difficulty (Dense)              (None, 3)            195         dense_8[0][0]                    
==================================================================================================
Total params: 1,286,984
Trainable params: 1,286,984
Non-trainable params: 0
__________________________________________________________________________________________________
```

모델 재활용은 기존에 매우 잘 훈련된 모델을 새로운 문제를 해결하는 딥러닝 신경망 모델을 
훈련시킬 때 많이 사용하는 기법이다.
{numref}`%s장 <ch:computer-vision-intro>`에서 전이학습을 소개하면서 모델 재활용 방식을 
자세히 다룬다.

## 신경망 모델 구성법 3: 서브클래싱

`keras.Model` 클래스를 상속하는 모델 클래스를 직접 선언하여 활용하는 기법이
**서브클래싱**<font size='2'>subclassing</font>이다.

서브클래싱으로 모델 클래서를 선언하려면 아래 두 메서드를 목적에 맞추어 
재정의<font size='2'>overriding</font>한다.

- `__init__()` 메서드(생성자): 은닉층과 출력층으로 사용될 층 객체 지정
- `call()` 메서드: 층을 연결하는 과정 지정. 즉, 입력값으부터 출력값을 만들어내는 순전파 과정 묘사.

앞서 함수형 API로 구성한 고객 요구사항 접수 모델(두 종류의 출력값 생성 모델)을 서브클래싱을 기법을 이용하여 구현하면 다음과 같다.

```python
class CustomerTicketModel(keras.Model):

    def __init__(self, num_departments): # num_departments: 고객 요구사항 처리 부서 개수
        super().__init__()
        self.concat_layer = layers.Concatenate()                      # 은닉층
        self.mixing_layer = layers.Dense(64, activation="relu")       # 은닉층
        self.priority_scorer = layers.Dense(1, activation="sigmoid")  # 출력층 1
        self.department_classifier = layers.Dense(                    # 출력층 2
            num_departments, activation="softmax")

    def call(self, inputs):               # inputs: 사전 객체 입력값. 모양은 미정.
        title = inputs["title"]
        text_body = inputs["text_body"]
        tags = inputs["tags"]

        features = self.concat_layer([title, text_body, tags])    # 은닉층 연결
        features = self.mixing_layer(features)
        priority = self.priority_scorer(features)                 # 출력층 연결
        department = self.department_classifier(features)
        return priority, department                               # 두 종류의 출력값 지정
```

이전과 동일한 구성의 모델은 다음과 같이 선언한다.

```python
>>> model = CustomerTicketModel(num_departments=4)
```

입력값의 정보가 없기에 층별 가중치와 편향은 초기화되지 않는다.

```python
>>> model.weights
[]
```

컴파일, 훈련, 평가, 예측은 이전과 완전히 동일한 방식으로 실행된다.

**서브클래싱 기법의 장단점**

- 장점
    - `call()` 함수를 이용하여 층을 임의로 구성할 수 있다.
    - `for` 반복문 등 파이썬 프로그래밍 모든 기법을 적용할 수 있다.
- 단점
    - 모델 구성을 전적으로 책임져야 한다.
    - 모델 구성 정보가 `call()` 함수 외부로 노출되지 않아서
        앞서 보았던 그래프 표현을 사용할 수 없다. 

## 혼합 신경망 모델 구성법

모델 서브클래싱은 `keras.layers.Layer`를 상속하여 사용자 정의 층 클래스를 선언하는 방식과 
거의 동일하다 ({numref}`%s장 <ch:keras-tf>` 참고).
사실 `keras.Model`이 `keras.layers.Layer`의 자식 클래스이다.
다만 모델 클래스는 `fit()`, `evaluate()`, `predict()` 메서드를 함께 지원하기에 
모델에 포함된 층의 가중치 행렬과 편향 벡터를 활용하는 훈련, 평가, 예측을 총괄할 수 있을 뿐이다. 

모델 클래스가 층의 하위 클래스이기에 
선언된/훈련된 모델을 다른 모델의 하나의 층으로 활용할 수도 있다.
아래 두 예제는 이 사실을 이용하여 앞서 소개된 세 가지 방식을 임의로 혼합할 수 있음을 보여준다.

**예제: 서브클래싱 모델을 함수형 모델에 활용하기** (강추!!!)

먼저 서브클래싱으로 하나의 모델을 선언한다.

- 한 개의 밀집층만 사용
- 이진 분류인 경우: 하나의 유닛과 `sigmoid` 활성화 함수 사용
- 다중 클래스 분류인 경우: 클래스(범주) 수 만큼의 유닛과  `softmax` 활성화 함수 사용

```python
class Classifier(keras.Model):

    def __init__(self, num_classes=2):
        super().__init__()
        if num_classes == 2:
            num_units = 1
            activation = "sigmoid"
        else:
            num_units = num_classes
            activation = "softmax"
        self.dense = layers.Dense(num_units, activation=activation)

    def call(self, inputs):
        return self.dense(inputs)
```

아래 코드는 함수형 API 형식으로 새로운 모델을 지정한다.
이때 위 모델 클래스의 객체를 출력층으로 사용한다.

```python
inputs = keras.Input(shape=(3,))                       # 입력층
features = layers.Dense(64, activation="relu")(inputs) # 은닉층
outputs = Classifier(num_classes=10)(features)         # 출력층

model = keras.Model(inputs=inputs, outputs=outputs)
```

**예제: 함수형 모델을 서브클래싱 모델에 활용하기**

먼저 함수형 API 형식으로 하나의 모델을 선언한다.

```python
inputs = keras.Input(shape=(64,))
outputs = layers.Dense(1, activation="sigmoid")(inputs)
binary_classifier = keras.Model(inputs=inputs, outputs=outputs)
```

아래 `MyModel` 클래스는 앞서 선언한 `binary_classifier` 모델을 출력층으로 활용한다.

```python
class MyModel(keras.Model):

    def __init__(self, num_classes=2):
        super().__init__()
        self.dense = layers.Dense(64, activation="relu")
        self.classifier = binary_classifier

    def call(self, inputs):
        features = self.dense(inputs)
        return self.classifier(features)
```

## 훈련 평가 방식 지정

딥러닝 신경망 모델의 훈련은 한 번 시작되면 훈련이 종료될 때까지 어떤 간섭도 받지 않는다.
즉, 훈련 시작 이전의 컴파일 과정에서 지정된 설정을 훈련 중간에 변경할 수 없다.
다만, 훈련 진행과정을 관찰<font size='2'>monitoring</font>할 수 있을 뿐이다.

훈련 과정 동안 관찰할 수 있는 내용은 일반적으로 다음과 같다.

- 에포크별 손실값
- 에포크별 평가지표

손실값과 평가지표를 계산하는 방식은 모델을 컴파일할 때 지정된다.
케라스가 지원하는 방식 중에 하나를 선택하거나 
필요에 따라 사용자가 직접 선언한 손실함수 또는 평가지표 클래스를 활용할 수 있다.
평가지표의 경우 `keras.metrics.Metric` 클래스를 상속해야 케라스의 다른 평가지표 클래스와 호환된다.

### 사용자 정의 평가지표

사용자가 직접 평가지표를 정의하려면 `Metric` 클래스를 상속하는 클래스를 선언해야 한다.
특히 아래 세 개의 메서드를 재정의<font size='2'>overriding</font> 한다.

- `update_state()`
- `result()`
- `reset_state()`

**`RootMeanSquaredError` 클래스 선언**

 평균 제곱근 오차(RMSE)를 평가지표로 사용하는 객체를 생성한다.

```python
class RootMeanSquaredError(keras.metrics.Metric):

    def __init__(self, name="rmse", **kwargs):
        super().__init__(name=name, **kwargs)
        self.mse_sum = self.add_weight(name="mse_sum", initializer="zeros")
        self.total_samples = self.add_weight(
            name="total_samples", initializer="zeros", dtype="int32")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.one_hot(y_true, depth=tf.shape(y_pred)[1])
        mse = tf.reduce_sum(tf.square(y_true - y_pred))
        self.mse_sum.assign_add(mse)
        num_samples = tf.shape(y_pred)[0]
        self.total_samples.assign_add(num_samples)

    def result(self):
        return tf.sqrt(self.mse_sum / tf.cast(self.total_samples, tf.float32))

    def reset_state(self):
        self.mse_sum.assign(0.)
        self.total_samples.assign(0)
```

포함된 메서드를 하나씩 살펴 보자.

- 생성자: `Metric` 클래스 상속, 평가지표 이름 지정, 에포크 별로 계산될 평가지표 계산에 필요한 속성(변수) 초기화
    - `name="rmse"`: 평가지표 이름. 훈련중에 평가지표 구분에 사용됨.
    - `mse_sum`: (에포크 별) 누적 제곱 오차. 0으로 초기화.
    - `total_samples`: (에포크 별) 훈련에 사용된 총 데이터 샘플 수. 0으로 초기화.

```python
def __init__(self, name="rmse", **kwargs):
    super().__init__(name=name, **kwargs)
    self.mse_sum = self.add_weight(name="mse_sum", initializer="zeros")
    self.total_samples = self.add_weight(
        name="total_samples", initializer="zeros", dtype="int32")
```

- `update_state`: 에포크 내에서 스텝 단위로 지정된 속성 업데이트
    - `mse`: 입력 배치 단위로 계산된 모든 샘플들에 대한 예측 오차의 제곱의 합.
    - `mse_sum` 업데이트: 새롭게 계산된 `mse`를 기존 `mse_sum`에 더함.
    - `num_samples`: 배치 크기
    - `total_samples` 업데이트: 새로 훈련된 배치 크기를 기존 `total_samples`에 더함

```python
def update_state(self, y_true, y_pred, sample_weight=None):
    y_true = tf.one_hot(y_true, depth=tf.shape(y_pred)[1])
    mse = tf.reduce_sum(tf.square(y_true - y_pred))
    self.mse_sum.assign_add(mse)
    num_samples = tf.shape(y_pred)[0]
    self.total_samples.assign_add(num_samples)
```

- `result`: 에포크 별로 평가지표 계산
    - 여기서는 에포크 별로 평균 제곱근 오차 계산.
    - `mse_sum`을 `total_samlpes`으로 나눈 값의 제곱근 계산

```python
def result(self):
    return tf.sqrt(self.mse_sum / tf.cast(self.total_samples, tf.float32))
```

- `reset_state`: 새로운 에포크 훈련이 시작될 때 모든 인스턴스 속성(변수) 초기화
    - 여기서는 `mse_sum`과 `total_samlpes` 모두 0으로 초기화

```python
def reset_state(self):
    self.mse_sum.assign(0.)
    self.total_samples.assign(0)
```

**사용자 정의 평가지표 활용**

직접 정의한 평가지표를 활용하려면 직접 해당 클래스의 객제를 지정하면 된다.

```python
model = get_mnist_model()
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy", RootMeanSquaredError()])
```

훈련을 시작하면 지정된 평가지표 이름과 함께 에포크 단위로 계산된 평가지표 값이 다른 평가지표, 손실값 등과 함께 출력된다.

```python
>>> model.fit(train_images, train_labels,
              epochs=3,
              validation_data=(val_images, val_labels))
```

```
Epoch 1/3
1563/1563 [==============================] - 12s 7ms/step - loss: 0.2929 - accuracy: 0.9136 - rmse: 7.1776 - val_loss: 0.1463 - val_accuracy: 0.9576 - val_rmse: 7.3533
Epoch 2/3
1563/1563 [==============================] - 11s 7ms/step - loss: 0.1585 - accuracy: 0.9546 - rmse: 7.3526 - val_loss: 0.1215 - val_accuracy: 0.9650 - val_rmse: 7.3954
Epoch 3/3
1563/1563 [==============================] - 11s 7ms/step - loss: 0.1293 - accuracy: 0.9636 - rmse: 7.3837 - val_loss: 0.1058 - val_accuracy: 0.9711 - val_rmse: 7.4182
313/313 [==============================] - 2s 5ms/step - loss: 0.1003 - accuracy: 0.9731 - rmse: 7.4307
```

이제 훈련 과정에서 계산되는 손실값과 평가지표를 활용하는 방식을 지정하는 콜백 기능과
텐서보드를 소개한다.

### 콜백

컴퓨터 프로그래밍에서 **콜백**<font size='2'>callback</font>은
하나의 프로그램이 실행되는 도중에 추가적으로 다른 API를 호출하는 기능 또는 해당 API를 가리킨다.
호출된 콜백은 자신을 호출한 프로그램과 독립적으로 실행된다.

신경망 모델 훈련 과정에서 사용될 수 있는 다양한 콜백 기능을 케라스가 제공한다.
일반적으로 손실값, 평가지표 등 훈련 중에 계산되는 기록을 저장하여 활용하며,
가장 많이 활용되는 콜백은 다음과 같다.

- 훈련 기록 작성
    - 훈련셋, 검증셋에 대한 손실값, 평가지표 기록 및 출력
    - 훈련 에포크마다 보여지는 손실값, 평가지표 등 관리
    - `keras.callbacks.CSVLogger` 클래스 활용. 
    - 기본으로 지원되도록 설정되었음.
    
- 훈련중인 모델의 상태 저장
    - 모델의 **상태**: 훈련 중인 모델에 저장된 가중치와 편향 등의 파라미터 정보
    - 훈련 중 가장 좋은 성능의 모델(의 상태) 저장
    - `keras.callbacks.ModelCheckpoint` 클래스 활용

- 훈련 조기 종료
    - 검증셋에 대한 손실이 더 이상 개선되지 않는 경우 훈련을 종료 시키기
    - `keras.callbacks.EarlyStopping` 클래스 활용

- 하이퍼 파라미터 조정
    - 학습률 동적 변경 지원
    - `keras.callbacks.LearningRateScheduler` 또는 `keras.callbacks.ReduceLROnPlateau` 클래스 활용

**예제**

콜백은 `fit()` 함수의 `callbacks`라는 옵션 인자를 이용하여 지정한다.
예를 들어 아래 코드는 두 종류의 콜백을 지정한다.

- `EarlyStopping`: 검증셋에 대한 정확도가 2 에포크 연속 개선되지 않을 때 훈련 종료
- `ModelCheckpoint`: 매 에포크마다 훈련된 모델 상태 저장
    - `save_best_only=True`: 검증셋에 대한 손실값이 가장 낮은 모델만 저장

```python
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=2,
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="checkpoint_path",
        monitor="val_loss",
        save_best_only=True,
    )
]

model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
model.fit(train_images, train_labels,
          epochs=10,
          callbacks=callbacks_list,
          validation_data=(val_images, val_labels))
```

**사용자 정의 콜백**

케라스와 호환되는 콜백 클래스를 정의하려면 `keras.callbacks.Callback` 클래스를 상속하면 된다.
매 에포크와 매 배치 훈련 단계의 시작과 종료 지점에서
수행해야 할 기능을 정의해야 하며 아래 메서드를 재정의하는 방식으로 이루어진다.

```python
on_epoch_begin(epoch, logs)
on_epoch_end(epoch, logs)
on_batch_begin(batch, logs)
on_batch_end(batch, logs)
on_train_begin(logs)
on_train_end(logs)
```

각 메서드에 사용되는 인자는 훈련 과정 중에 자동으로 생성된 객체로부터 값을 받아온다.

- `logs` 인자: 이전 배치와 에포크의 훈련셋과 검증셋에 대한 손실값, 평가지표 등을 포함한 사전 객체.
- `batch`와 `epoch`: 배치와 에포크 정보

다음 `LossHistory` 콜백 클래스는 배치 훈련이 끝날 때마다 손실값을 저장하고
에포크가 끝날 때마다 배치별 손실값을 그래프로 저장하여 훈련이 종료된 후 시각화하여 보여주도록 한다.

```python
from matplotlib import pyplot as plt

class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs):
        self.per_batch_losses = []

    def on_batch_end(self, batch, logs):
        self.per_batch_losses.append(logs.get("loss"))

    def on_epoch_end(self, epoch, logs):
        plt.clf()
        plt.plot(range(len(self.per_batch_losses)), self.per_batch_losses,
                 label="Training loss for each batch")
        plt.xlabel(f"Batch (epoch {epoch})")
        plt.ylabel("Loss")
        plt.legend()
        plt.savefig(f"plot_at_epoch_{epoch}")
        self.per_batch_losses = []
```

### 텐서보드

**텐서보드**<font size='2'>TensorBoard</font>는 모델 훈련 과정을 시각적으로 실시간 관찰할 수 있는 도구를 제공하며
텐서플로우와 함께 설치된다.

- 신경망 모델 구조 시각화
- 손실값, 정확도 등의 변화 시각화
- 가중치, 편향 텐서 등의 변화 히스토그램
- 이미지, 텍스트, 오디오 데이터 시각화
- 기타 다양한 기능 제공

<div align="center"><img src="https://github.com/codingalzi/dlp2/blob/master/jupyter-book/imgs/ch07-tensorboard.png?raw=true" style="width:600px;"></div>

텐서보드는 `TensorBoard` 콜백 클래스를 활용한다.

- `log_dir`: 텐서보드 서버 실행에 필요한 데이터 저장소 지정

```python
tensorboard = keras.callbacks.TensorBoard(
    log_dir="./tensorboard_log_dir",
)

model.fit(train_images, train_labels,
          epochs=10,
          validation_data=(val_images, val_labels),
          callbacks=[tensorboard])
```

모델 훈련 중 또는 이후에 텐서보드 서버를 실행한다.

- 주피터 노트북에서

```python
%load_ext tensorboard
%tensorboard --logdir ./tensorboard_log_dir
```

- 터미널에서

```python
tensorboard --logdir ./tensorboard_log_dir
```

## 연습문제

1. [(실습) 케라스 신경망 모델 활용법](https://colab.research.google.com/github/codingalzi/dlp2/blob/master/excs/exc-working_with_keras.ipynb)